# Raster to Vector Conversion in Pixel Art Images

Steps:

0) Import libraries
1) Load a pixel art image
2) Create an adjacency graph
3) Simplify adjacency graph
4) Calculate and Render the Dual Graph
5) Rectify T Junctions
6) Interpolate and Render Splines
7) Smoothen the diagram

## 0) Import Libraries

In [1]:
# TODO (P1): Add a list of install functions, with versions, that one would need to run in a fresh environment

import cv2
import math
import numpy as np

import tkinter as tk
from tkinter import filedialog
from IPython.display import SVG, display, HTML

## 1) Load a Pixel Art Image

In [2]:
# TODO (P2): Add text explaining the variables

save_reduced_input = False
pixel_size = 20

In [3]:
def get_input_raster(verbose = True):
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select a PNG image",
        filetypes=[("PNG files", "*.png")]
    )
    if verbose:
        print("Selected file:", file_path)
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    input_raster = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
    return input_raster

In [4]:
input_art = get_input_raster()

Selected file: C:/Users/shrey/Documents/SVG Experiments/pixelralsei.png


In [5]:
# TODO (P1): Make this a method

# Generate a bare minimum pixel art image
pixel_size = 0

for row in range(input_art.shape[0]):
    curr_pixel_size = 1
    curr_colour = input_art[row,0]
    for col in range(1, input_art.shape[1]):
        pixel_color = input_art[row,col]
        if (pixel_color == curr_colour).all():
            curr_pixel_size += 1
        else:
            pixel_size = math.gcd(curr_pixel_size, pixel_size)
            curr_pixel_size = 1
            curr_colour = pixel_color

# TODO (P1): Do the same vertically as well

if pixel_size > 1:
    print(f'Each pixel in the image has a pixel size of {pixel_size}. Resizing the image')

pixel_art = np.zeros((input_art.shape[0]//pixel_size, input_art.shape[1]//pixel_size, 4), dtype=np.uint8)

for row in range(pixel_art.shape[0]):
    for col in range(pixel_art.shape[1]):
        pixel_art[row][col] = input_art[row*pixel_size][col*pixel_size]

In [6]:
# TODO (P1): Make this a method
if save_reduced_input:
    saved_art = cv2.cvtColor(pixel_art, cv2.COLOR_BGRA2RGBA)
    cv2.imwrite(file_path, saved_art)

In [7]:
def get_pixel_art_svg_For_image(pixel_art, pixel_size = 20):
    svg_elements = []
    for row in range(pixel_art.shape[0]):
        for col in range(pixel_art.shape[1]):
            svg_elements.append(f'<rect width="{pixel_size}" height="{pixel_size}" fill="rgba({pixel_art[row][col][0]}, {pixel_art[row][col][1]}, {pixel_art[row][col][2]}, {pixel_art[row][col][3]})" '
                                + f'transform="translate({col*pixel_size}, {row*pixel_size})"/>')
    return svg_elements

# TODO (P1): Provide a way to export the SVG taht can be opened in an SVG editor
# TODO (P3): Remove dependency on pixel_art. Find a better way to size the canvas
def wrap_svg_elements_under_html(svg_elements, pixel_art, pixel_size = 20):
    svg_code = f'''<svg width="{pixel_art.shape[1]*pixel_size}" height="{pixel_art.shape[0]*pixel_size}" xmlns="http://www.w3.org/2000/svg" style="background-color: transparent;">\n\t'''\
    + '\n\t'.join(svg_elements)\
    + '''</svg>'''

    html = '''<div style="background-color: transparent; padding: 0px;">\n''' + svg_code + '''</div>'''
    return html

In [8]:
pixel_svg_elements = get_pixel_art_svg_For_image(pixel_art)

html = wrap_svg_elements_under_html(pixel_svg_elements, pixel_art)

HTML(html)

## 2) Create an adjacency graph

The adjacency graph will be a 3D boolean adjacency matrix of shape $height \times width \times 8$ For the node $N$, the edges are denoted by the following indices

$$
\begin{pmatrix}
0 & 1 & 2 \\
3 & N & 4 \\
5 & 6 & 7
\end{pmatrix}
$$

In [9]:
# TODO (P1): Make the above description more detailed

In [10]:
from modules.pixel_adjacency_graph import PixelAdjacencyGraph

In [11]:
# TODO (P1): Encapsulate the below methods into a class

# Method to get a list of SVG objects containing nodes as circles
def get_adjacency_graph_node_svg_objects(adjacency_graph, pixel_size = 20, node_colour = (0, 255, 0, 0.33), node_colour_failure = (255, 0, 0, 1.0), node_radius_ratio = 0.2, mark_erroneous_nodes = True):
    adjacency_matrix = adjacency_graph.get_adjacency_matrix()
    if mark_erroneous_nodes:
        is_node_planar = adjacency_graph.get_non_planar_nodes()

    node_radius = pixel_size * node_radius_ratio
    svg_elements = []
    for row in range(adjacency_matrix.shape[0]):
        for col in range(adjacency_matrix.shape[1]):
            rendered_colour = node_colour
            if mark_erroneous_nodes and not is_node_planar[row, col]:
                rendered_colour = node_colour_failure
            svg_elements.append(f'<circle cx="{(col+0.5) * pixel_size}" cy="{(row+0.5) * pixel_size}" r="{node_radius}" fill="rgba{rendered_colour}"/>')
    return svg_elements

# Method to get a list of rendered SVG adjacency edges
def get_adjacency_graph_edge_svg_objects(adjacency_graph, pixel_size = 20, edge_colour = (0, 255, 0, 0.5), edge_colour_failure = (255, 0, 0, 1.0), line_width = 2, mark_erroneous_nodes = True):
    adjacency_matrix = adjacency_graph.get_adjacency_matrix()
    if mark_erroneous_nodes:
        is_node_planar = adjacency_graph.get_non_planar_nodes()
    
    svg_elements = []

    row_inc = [-1, -1, -1,  0,  0,  1,  1,  1]
    col_inc = [-1,  0,  1, -1,  1, -1,  0,  1]

    for row in range(adjacency_matrix.shape[0]):
        for col in range(adjacency_matrix.shape[1]):
            for i in range(4):
                if(adjacency_matrix[row, col, i]):
                    x1 = (col+0.5) * pixel_size
                    y1 = (row+0.5) * pixel_size
                    x2 = (col+col_inc[i]+0.5) * pixel_size
                    y2 = (row+row_inc[i]+0.5) * pixel_size
                    rendered_colour = edge_colour
                    if mark_erroneous_nodes\
                            and i in [0, 2]\
                            and not is_node_planar[row, col]\
                            and not is_node_planar[row+row_inc[i], col+col_inc[i]]\
                            and not is_node_planar[row, col+col_inc[i]]\
                            and not is_node_planar[row+row_inc[i], col]:
                        rendered_colour = edge_colour_failure
                    svg_elements.append(f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="rgba{rendered_colour}" stroke-width="{line_width}" />')
    return svg_elements

In [12]:
adjacency_graph = PixelAdjacencyGraph(pixel_art)

# Render the adjacency graph
pixel_svg_elements = get_pixel_art_svg_For_image(pixel_art)
node_svg_elements = get_adjacency_graph_node_svg_objects(adjacency_graph)
edge_svg_elements = get_adjacency_graph_edge_svg_objects(adjacency_graph)

html = wrap_svg_elements_under_html(pixel_svg_elements + node_svg_elements + edge_svg_elements, pixel_art)

HTML(html)

# 3) Simplify adjacency graph

In [13]:
#TODO

# 4) Calculate and Render the Dual Graph

In [14]:
#TODO

# 5) Rectify T Junctions

In [15]:
#TODO

# 6) Interpolate and Render Splines

In [16]:
#TODO

# 7) Smoothen the diagram

In [17]:
#TODO